In [ ]:
import pandas as pd, os
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
# from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import time
from sklearn.metrics import accuracy_score
# model_type = 'LSTM'
model_type = 'MLP'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# df = pd.read_csv('../lstm covid19/dataset2/SARS-CoV-2_2349_50up.csv')
# df = pd.read_csv('../lstm covid19/dataset2/SARS-CoV-2_2262_100up_time.csv')
# columnsNames = df.columns.values
# columnsNames = [i+'_DNA' for i in columnsNames]
# df.columns=columnsNames
# df_text = df.rename(columns={'DNA_lineage':'lineage', 'DNA_time':'time'})
# y_text = [i[0] for i in np.array(df_text)]
# X_text = [i[1::] for i in np.array(df_text)]

In [ ]:
# label_text = np.unique(y_text, return_counts=False,return_index=False,return_inverse=False)
# feature_text = np.unique(X_text, return_counts=False,return_index=False,return_inverse=False)
# print(label_text, feature_text)

In [ ]:
import pandas as pd,os
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
# from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import time
from sklearn.metrics import accuracy_score
# model_type = 'LSTM'
model_type = 'MLP'
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
# X_train, ans_train = np.load('../lstm covid19/dataset2/numpy_train_test_split_added time/X_train_100up_weeks.npy', allow_pickle=True), np.load('../lstm covid19/dataset2/numpy_train_test_split_added time/y_train_100up_weeks.npy', allow_pickle=True)

# X_test, ans_test = np.load('../lstm covid19/dataset2/numpy_train_test_split_added time/X_test_100up_weeks.npy', allow_pickle=True), np.load('../lstm covid19/dataset2/numpy_train_test_split_added time/y_test_100up_weeks.npy', allow_pickle=True)
# print(X_train.shape, ans_train.shape, X_test.shape, ans_test.shape)

X_train, ans_train = np.load('./dataset3/X_train_class100_time.npy'), np.load('./dataset3/y_train_class100_time.npy', allow_pickle=True)

X_test, ans_test = np.load('./dataset3/X_test_class100_time.npy', allow_pickle=True), np.load('./dataset3/y_test_class100_time.npy', allow_pickle=True)
print(X_train.shape, ans_train.shape, X_test.shape, ans_test.shape)

In [ ]:
label_train = np.unique(ans_train, return_counts=False,return_index=False,return_inverse=False)
label_test = np.unique(ans_test, return_counts=False,return_index=False,return_inverse=False)
print(label_train, label_test)

In [ ]:
def label2value(data):
    y = []
    for i in data:
        for id, j in enumerate(label_test):
            if j==i:
                y.append(id)
    return np.array(y)
y_train = label2value(ans_train)
y_test = label2value(ans_test)
print(y_train.shape, y_test.shape)

In [ ]:
import torch.utils.data as data
batch_size = 128
train_zip = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
test_zip = TensorDataset(torch.tensor(X_test), torch.tensor(y_test))

train_loader = DataLoader(dataset=train_zip, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=test_zip, batch_size=batch_size, shuffle=False)

In [ ]:
def accuracy(predictions, labels):
    classes = torch.argmax(predictions, dim=1)
    return torch.mean((classes == labels).float())

In [ ]:
class RNN(nn.Module):
    def __init__(self, class_n, Input_Size):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(
            input_size=Input_Size,
            hidden_size=128,
            num_layers=1,
            batch_first=True,
        )

        self.out = nn.Linear(128, class_n)
    def forward(self, x):
        r_out, (h_c, h_h) = self.rnn(x, None)
        out = self.out(r_out[:, -1, :])
        return out
    
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.normal_(m.weight.data, 0, 0.01)
                # m.weight.data.normal_(0,0.01)
                m.bias.data.zero_()

In [ ]:
class MLP(nn.Module):
    def __init__(self, num_classes, input_size):
        super(MLP,self).__init__()
        self.linear1 = nn.Linear(in_features=input_size, out_features=1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.dt1 = nn.Dropout(0.25)
        self.linear2 = nn.Linear(in_features=1024, out_features=256)
        self.bn2 = nn.BatchNorm1d(256)
        self.dt2 = nn.Dropout(0.25)
        self.linear3 = nn.Linear(in_features=256, out_features= num_classes)
        
    def forward(self, x):
        x = self.bn1(self.linear1(x))
        x = F.relu(x)
        # x = self.dt1(x)
        x = self.bn2(self.linear2(x))
        x = F.relu(x)
        # x = self.dt2(x)
        x = self.linear3(x)
        x = F.softmax(x, dim=1)
        return x
        
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                torch.nn.init.normal_(m.weight.data, 0, 0.01)
                # m.weight.data.normal_(0,0.01)
                m.bias.data.zero_()

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
if hasattr(torch.cuda, 'empty_cache'):
        torch.cuda.empty_cache()
Input_Size = np.array(X_train).shape[1]
min_valid_loss = np.inf
if model_type=='LSTM':
        model = RNN(max(y_train)+1, Input_Size).to(device)
else:
        model = MLP(max(y_train)+1, Input_Size).to(device)

model.initialize_weights()

LR = 0.0001
loss_fun = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
# Starting trainer
for epoch in tqdm(range(1000)):
        for step, (b_x, b_y) in enumerate(train_loader):
            # Forward pass
            model.train()
            train_loss = 0.0
            # if torch.cuda.is_available():
            #     if model_type=='LSTM':
            #         b_x_, b_y_= (torch.unsqueeze(b_x, 1)).data.type(torch.FloatTensor).to(device), b_y.to(device)
            #     else:
            #         b_x_, b_y_= b_x.float().to(device), b_y.to(device)
            # else:
            #     if model_type=='LSTM':
            #         b_x_, b_y_= (torch.unsqueeze(b_x, 1)).data.type(torch.FloatTensor).to(device), b_y.to(device)
            #     else:
            #         b_x_, b_y_= b_x.float().to(device), b_y.to(device)
            if model_type=='LSTM':
                b_x_, b_y_= (torch.unsqueeze(b_x, 1)).data.type(torch.FloatTensor).to(device), b_y.to(device)
            else:
                b_x_, b_y_= b_x.float().to(device), b_y.to(device)
            optimizer.zero_grad()
            r_out = model(b_x_)
            loss = loss_fun(r_out, b_y_)
            loss.backward()
            optimizer.step()
            train_loss = loss.item()*b_x_.size(0)
            del b_x_, b_y_
            torch.cuda.empty_cache()

        # Validation
        with torch.no_grad():
            valid_loss = 0.0
            model.eval()
            for step, (b_x, b_y) in enumerate(valid_loader):
                if model_type=='LSTM':
                    b_x_, b_y_= (torch.unsqueeze(b_x, 1)).data.type(torch.FloatTensor).to(device), b_y.to(device)
                else:
                    b_x_, b_y_= b_x.float().to(device), b_y.to(device)
                # if torch.cuda.is_available():
                #     if model_type=='LSTM':
                #         b_x_, b_y_= (torch.unsqueeze(b_x, 1)).data.type(torch.FloatTensor).to(device), b_y.to(device)
                #     else:
                #         b_x_, b_y_= b_x.float().to(device), b_y.to(device)
                r_out = model(b_x_)
                loss = loss_fun(r_out, b_y_)
                valid_loss = loss.item()*b_x_.size(0)
                del b_x_, b_y_
                torch.cuda.empty_cache()

            if min_valid_loss > valid_loss:
                # print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
                min_valid_loss = valid_loss
                
                # Saving State Dict
                torch.save({'state_dict': model.state_dict()}, f'./pth/{model_type}_saved_model_50_epoch-1000.pth.tar')

In [ ]:
# model loadings
if model_type=='LSTM':
        model = RNN(max(y_train)+1, Input_Size).to(device)
else:
        model = MLP(max(y_train)+1, Input_Size).to(device)
# checkpoint = torch.load(f'./pth/{model_type}_saved_model_50_epoch-1000.pth.tar', map_location='cpu')
checkpoint = torch.load(f'./pth/{model_type}_saved_model_50_epoch-1000.pth.tar')
model.load_state_dict(checkpoint['state_dict'])
print(model)

In [ ]:
DNA_list = ['-', 'A', 'C', 'G', 'N', 'T']
print(X_train.shape[1])

In [ ]:
fn = pd.read_csv('../lstm covid19/feature_name.csv')

fn = np.array(fn.values)
fn = fn.flatten()

In [ ]:
# def index2text(value_data):
#     text=[]
#     for id1, n1 in enumerate(np.array(value_data)):
#         for id2, n2 in enumerate(feature_text):
#             if n1==id2:
#                 add_ = str(n2)+'_'+str(id1)
#                 text.append(add_)
#     return text
import matplotlib.pyplot as plt
import shap
shap.initjs()
# class_n = 3 #0-7
# for f_id, class_n in enumerate(label_train):
f_id = 0
class_n = label_test[f_id]
batch = next(iter(valid_loader))
value, ans = batch
new_value = []
# for id, i in enumerate(ans):
    # if i==f_id:
        # new_value.append(np.array(value[id]))

# background = torch.tensor(new_value)
background = value
if model_type=='LSTM':
    background_ex = (torch.unsqueeze(background, 1)).data.type(torch.FloatTensor)
else:
    background_ex = background.data.type(torch.FloatTensor)

print('Test Data Shape ',value.shape)
print('Extract Data Shape ', background.shape)
print('Squeeze Data Shape ', background_ex.shape)

with torch.no_grad():
    e = shap.DeepExplainer(model.cpu(), background_ex.cpu())
shap_values = e.shap_values(background_ex.cpu())

test_stack = []
# for i in range(len(new_value)):
#     temp = index2text(new_value[i])
#     test_stack.append(temp)
test_stack = np.array(value)
# test_stack = np.array(test_stack)
print('Ground Turth Data Shape ', test_stack.shape)
if model_type=='LSTM':
    shap_values_ = np.squeeze(np.array(shap_values),axis=-2)
else:
    shap_values_ = np.array(shap_values)

print('Shap Values Shape ', shap_values_.shape)



In [ ]:
f_id = 6
class_n = label_test[f_id]
print(shap_values_[f_id].shape, test_stack.shape)
# for f_id, class_n in enumerate(label_train):
shap.summary_plot(list(shap_values_), test_stack, plot_type="bar", class_names=label_test, feature_names=fn, show=False)
fig = plt.gcf()
fig.set_figheight(12)
fig.set_figwidth(14)
ax = plt.gca()
ax.set_ylabel('Feature', fontsize=16)
leg = ax.legend()

plt.title(f'{model_type} Multiclass', fontsize=20)
# plt.show()
plt.rcParams["figure.figsize"] = (10,20)
plt.savefig(f'/ssd1/報告//Images//DNA/{model_type}//{model_type} multiclass-100-output.jpg', dpi=300)
# shap.summary_plot(shap_values, X)

In [ ]:
np.array(shap_values[1])

In [ ]:
shap_values[0]

In [ ]:
shap_values[1]